In [ ]:
# 1. Instala a biblioteca (caso necessário)
!pip install google-cloud-bigquery
!pip install seaborn
!pip install matplotlib
!pip install pandas
!pip install numpy
!pip install requests beautifulsoup4 pandas

In [ ]:
# 2. Autenticação
from google.colab import auth
auth.authenticate_user()

In [ ]:
# 3. Conecta ao BigQuery
from google.cloud import bigquery
client = bigquery.Client(project="my-project-laboratoria")

# 4. Consulta a VIEW no BigQuery
query = """
SELECT
*
FROM `my-project-laboratoria.project_rota01.superstore_corrigido`
"""

# 5. Executa a query e mostra os dados como DataFrame
df = client.query(query).to_dataframe()
df.head()

,category,city,country,customer_name,market,market2,order_priority,product_name,region,segment,...,unknown,order_date,profit,quantity,row_id,sales,ship_date,shipping_cost,year,weeknum
0,Office Supplies,Aba,Nigeria,Dianna Wilson,Africa,Africa,Medium,"Boston Pencil Sharpener, Water Color",Africa,Home Office,...,1,2011-04-16 00:00:00+00:00,-90.174,14,43411,135,2011-04-21 00:00:00+00:00,10.61,2011,16
1,Office Supplies,Pescara,Italy,Alan Haines,Eu,Eu,Medium,"Binney & Smith Pencil Sharpener, Water Color",South,Corporate,...,1,2011-04-08 00:00:00+00:00,49.560,4,19231,108,2011-04-12 00:00:00+00:00,9.00,2011,15
2,Furniture,Pescara,Italy,Alan Haines,Eu,Eu,Medium,"Tenex Light Bulb, Durable",South,Corporate,...,1,2011-04-08 00:00:00+00:00,7.560,4,19230,64,2011-04-12 00:00:00+00:00,3.57,2011,15
3,Office Supplies,Pescara,Italy,Rick Huthwaite,Eu,Eu,Medium,"Rogers File Cart, Industrial",South,Home Office,...,1,2011-09-30 00:00:00+00:00,-32.580,1,11481,85,2011-10-04 00:00:00+00:00,5.09,2011,40
4,Office Supplies,Banda Aceh,Indonesia,Maureen Fritzler,Apac,Apac,Medium,"Stanley Pens, Easy-Erase",Southeast Asia,Corporate,...,1,2011-03-25 00:00:00+00:00,4.068,5,26626,38,2011-03-29 00:00:00+00:00,2.52,2011,13


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   category        51290 non-null  object             
 1   city            51290 non-null  object             
 2   country         51290 non-null  object             
 3   customer_name   51290 non-null  object             
 4   market          51290 non-null  object             
 5   market2         51290 non-null  object             
 6   order_priority  51290 non-null  object             
 7   product_name    51290 non-null  object             
 8   region          51290 non-null  object             
 9   segment         51290 non-null  object             
 10  ship_mode       51290 non-null  object             
 11  state           51290 non-null  object             
 12  sub_category    51290 non-null  object             
 13  customer_ID     51290 non-null 

In [ ]:
import pandas as pd
import numpy as np

# A sua linha de código para carregar o DataFrame
# Certifique-se de que a sua query seleciona as variáveis numéricas necessárias
df = client.query(query).to_dataframe()

# Lista das variáveis numéricas a serem analisadas
# Adapte esta lista se as suas colunas numéricas forem diferentes
variaveis_numericas = [
    'discount', 'profit', 'quantity', 'unknown', 'row_id',
    'sales', 'shipping_cost', 'year', 'weeknum'
]

print("Identificando valores discrepantes (outliers) usando o método IQR:")
print("-" * 70)

# Loop para identificar outliers em cada variável
for col in variaveis_numericas:
    # Verifica se a coluna existe no DataFrame
    if col not in df.columns:
        print(f"Aviso: A coluna '{col}' não foi encontrada no DataFrame. Pulando...")
        continue

    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR

    # Encontra os valores que estão fora dos limites
    outliers = df[(df[col] < limite_inferior) | (df[col] > limite_superior)]

    print(f"Variável: {col}")
    print(f"  Limites (IQR): [{limite_inferior:.2f}, {limite_superior:.2f}]")
    print(f"  Total de outliers encontrados: {len(outliers)}")

    if not outliers.empty:
        print("  Valores discrepantes:")
        print(outliers[[col]])
    else:
        print("  Nenhum outlier encontrado.")

    print("-" * 70)

Identificando valores discrepantes (outliers) usando o método IQR:
----------------------------------------------------------------------
Variável: discount
  Limites (IQR): [-0.30, 0.50]
  Total de outliers encontrados: 4172
  Valores discrepantes:
       discount
0           0.7
10          0.7
11          0.7
12          0.7
13          0.6
...         ...
51241       0.7
51286       0.7
51287       0.7
51288       0.7
51289       0.7

[4172 rows x 1 columns]
----------------------------------------------------------------------
Variável: profit
  Limites (IQR): [-55.22, 92.03]
  Total de outliers encontrados: 9755
  Valores discrepantes:
        profit
0      -90.174
10     -62.118
14     -83.478
15     -67.176
23     -55.992
...        ...
51261  629.280
51280  125.940
51283  137.160
51284  263.640
51288 -103.158

[9755 rows x 1 columns]
----------------------------------------------------------------------
Variável: quantity
  Limites (IQR): [-2.50, 9.50]
  Total de outliers enco

In [ ]:
import pandas as pd
import requests
import os

url = 'https://en.wikipedia.org/wiki/List_of_supermarket_chains'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# --- Configurações de Salvar CSV ---
# Nome do arquivo CSV de saída
nome_arquivo_csv = 'supermarket_chains.csv'
# Onde o arquivo será salvo (o diretório atual do script)
caminho_completo_csv = os.path.join(os.getcwd(), nome_arquivo_csv)
# ----------------------------------

try:
    print(f"Buscando dados em: {url}")
    response = requests.get(url, headers=headers)
    response.raise_for_status()

    # Usando o pandas para ler todas as tabelas na página
    tabelas = pd.read_html(response.content)

    # AQUI ESTÁ A CORREÇÃO ORIGINAL: Selecione a tabela no índice 1
    # Note que o índice da tabela pode mudar se a página da Wikipedia for atualizada
    tabela_supermercados = tabelas[1]

    print("Tabela de supermercados extraída com sucesso!")
    print(tabela_supermercados.head())

    # --- PROCESSO DE SALVAR EM CSV ---
    # Salva o DataFrame no arquivo CSV, sem o índice (index=False)
    tabela_supermercados.to_csv(nome_arquivo_csv, index=False, encoding='utf-8')

    print("-" * 30)
    print(f"SUCESSO! O DataFrame foi salvo como: {nome_arquivo_csv}")
    print(f"Caminho: {caminho_completo_csv}")
    print("-" * 30)
    # ---------------------------------

except requests.exceptions.RequestException as e:
    print(f"Ocorreu um erro na requisição: {e}")
except IndexError:
    print("A tabela não foi encontrada no índice esperado (1). O índice pode ter mudado.")
except Exception as e:
    print(f"Ocorreu um erro inesperado: {e}")

Buscando dados em: https://en.wikipedia.org/wiki/List_of_supermarket_chains
Tabela de supermercados extraída com sucesso!
          Company            Headquarters  \
0        7-Eleven     Japan United States   
1            Aeon                   Japan   
2  Ahold Delhaize             Netherlands   
3            Aldi  Germany (Süd and Nord)   
4           Edeka                 Germany   

         Served countries (besides the headquarters)  Map  \
0  Australia, Canada, China, Cambodia, Denmark, H...  NaN   
1  Australia, Cambodia, China, Hong Kong, India, ...  NaN   
2  Belgium (as Albert Heijn and Delhaize), Czech ...  NaN   
3  Australia (Süd), Austria (Süd as Hofer), China...  NaN   
4                                            Denmark  NaN   

                         Number of locations Number of employees  
0                                      85000              170000  
1                                      20008            560,000+  
2                                      

In [ ]:
# 0.1 Instalar e Importar Bibliotecas
# Instala a biblioteca do BigQuery no ambiente Colab
!pip install pandas google-cloud-bigquery

# Importa as bibliotecas necessárias
import pandas as pd
from google.cloud import bigquery
from google.colab import files
from google.colab import drive
import numpy as np

print("Bibliotecas instaladas e importadas com sucesso!")

Bibliotecas instaladas e importadas com sucesso!


In [ ]:
# 0.2 Autenticação no Google Cloud
from google.colab import auth
auth.authenticate_user()
print("Usuário autenticado no Google Cloud!")

Usuário autenticado no Google Cloud!


In [ ]:
# 0.3 Configuração do BigQuery
# --- CONFIGURAÇÕES DE DESTINO ---
# SUBSTITUA PELO SEU PROJECT ID do Google Cloud
PROJECT_ID = "my-project-laboratoria"
# SUBSTITUA PELO NOME DO SEU DATASET no BigQuery
DATASET_ID = "project_rota01"
# -------------------------------

client = bigquery.Client(project=PROJECT_ID)
print(f"Conexão com BigQuery configurada para o Projeto: {PROJECT_ID}")

Conexão com BigQuery configurada para o Projeto: my-project-laboratoria


In [ ]:
# 0.4 Extração (E) - Leitura da VIEW no BigQuery (CORRIGIDO)
# --- EXTRAÇÃO (E) - Lendo dados da VIEW Corrigida no BigQuery ---

query = """
SELECT
    *
FROM `my-project-laboratoria.project_rota01.superstore_corrigido`
"""

try:
    # 5. Executa a query e carrega os dados no DataFrame
    df_superstore = client.query(query).to_dataframe()

    print(f"Dados extraídos do BigQuery com sucesso. {len(df_superstore)} linhas.")

    # Exibe as primeiras linhas para confirmação
    print("Primeiras linhas do DataFrame 'df_superstore':")
    print(df_superstore.head(2).to_markdown(index=False))

except Exception as e:
    print(f"Ocorreu um erro ao consultar o BigQuery: {e}")
    # Cria um DataFrame vazio em caso de erro para evitar quebra do script
    df_superstore = pd.DataFrame()

Dados extraídos do BigQuery com sucesso. 51290 linhas.
Primeiras linhas do DataFrame 'df_superstore':
| category        | city    | country   | customer_name   | market   | market2   | order_priority   | product_name                                 | region   | segment     | ship_mode      | state   | sub_category   | customer_ID   | order_id        | product_id       |   discount |   unknown | order_date                |   profit |   quantity |   row_id |   sales | ship_date                 |   shipping_cost |   year |   weeknum |
|:----------------|:--------|:----------|:----------------|:---------|:----------|:-----------------|:---------------------------------------------|:---------|:------------|:---------------|:--------|:---------------|:--------------|:----------------|:-----------------|-----------:|----------:|:--------------------------|---------:|-----------:|---------:|--------:|:--------------------------|----------------:|-------:|----------:|
| Office Supplies | Aba   

## Passo 1: Transformação (T) - Limpeza e Dimensão Tempo

In [ ]:

# --- TRANSFORMAÇÃO (T) ---

# 1. TRATAMENTO INICIAL DE DATAS
# Converte as colunas de data para o tipo datetime
df_superstore['order_date'] = pd.to_datetime(df_superstore['order_date'], errors='coerce')
df_superstore['ship_date'] = pd.to_datetime(df_superstore['ship_date'], errors='coerce')

# 2. CRIAÇÃO DA DIMENSÃO TEMPO (DIM_TIME)
# Obtém todas as datas únicas de pedido e envio
datas_unicas = pd.concat([df_superstore['order_date'], df_superstore['ship_date']]).unique()
datas_unicas = pd.Series(datas_unicas).dropna().sort_values().reset_index(drop=True)

# Cria o DataFrame da Dimensão Tempo
df_time = pd.DataFrame({'data_completa': datas_unicas})

# Cria atributos de tempo
df_time['ano'] = df_time['data_completa'].dt.year
df_time['mes'] = df_time['data_completa'].dt.month
df_time['nome_mes'] = df_time['data_completa'].dt.strftime('%B')
df_time['trimestre'] = 'Q' + ((df_time['mes'] - 1) // 3 + 1).astype(str)
df_time['dia_da_semana'] = df_time['data_completa'].dt.dayofweek + 1 # 1=Segunda, 7=Domingo
df_time['semana_do_ano'] = df_time['data_completa'].dt.isocalendar().week.astype(int)

# Geração da CHAVE SUBSTITUTA (PK) no formato AAAAMMDD
df_time['chave_tempo'] = df_time['data_completa'].dt.strftime('%Y%m%d').astype(int)

# Seleciona as colunas finais
df_dim_tempo = df_time[['chave_tempo', 'data_completa', 'ano', 'mes', 'nome_mes', 'trimestre', 'dia_da_semana', 'semana_do_ano']]

print(f"Dim_Tempo criada com sucesso. {len(df_dim_tempo)} datas únicas.")
df_dim_tempo.head(2)

Dim_Tempo criada com sucesso. 1468 datas únicas.


,chave_tempo,data_completa,ano,mes,nome_mes,trimestre,dia_da_semana,semana_do_ano
0,20110101,2011-01-01 00:00:00+00:00,2011,1,January,Q1,6,52
1,20110102,2011-01-02 00:00:00+00:00,2011,1,January,Q1,7,52


## Passo 2: Transformação (T) - Dimensões Geográficas, Produto e Cliente

In [ ]:

# 2.1 Dimensões Geográfica e Produto
# 3. CRIAÇÃO DA DIMENSÃO LOCALIDADE (DIM_GEOGRAPHY)
cols_geo = ['city', 'state', 'country', 'region', 'market']
df_dim_localidade = df_superstore[cols_geo].drop_duplicates().reset_index(drop=True)

# Geração da CHAVE SUBSTITUTA (PK)
df_dim_localidade.insert(0, 'chave_localidade', df_dim_localidade.index + 1)

print(f"Dim_Localidade criada com sucesso. {len(df_dim_localidade)} localidades únicas.")
df_dim_localidade.head(2)

# 4. CRIAÇÃO DA DIMENSÃO PRODUTO (DIM_PRODUCT)
cols_prod = ['product_id', 'product_name', 'category', 'sub_category']
df_dim_produto = df_superstore[cols_prod].drop_duplicates(subset=['product_id']).reset_index(drop=True)

# Geração da CHAVE SUBSTITUTA (PK)
df_dim_produto.insert(0, 'chave_produto', df_dim_produto.index + 1)

print(f"Dim_Produto criada com sucesso. {len(df_dim_produto)} produtos únicos.")
df_dim_produto.head(2)

Dim_Localidade criada com sucesso. 3819 localidades únicas.
Dim_Produto criada com sucesso. 10292 produtos únicos.


,chave_produto,product_id,product_name,category,sub_category
0,1,OFF-BOS-10001348,"Boston Pencil Sharpener, Water Color",Office Supplies,Art
1,2,OFF-AR-10002382,"Binney & Smith Pencil Sharpener, Water Color",Office Supplies,Art


## 2.2 Dimensão Cliente (Simulação Inicial SCD Tipo 2)

In [ ]:

# 5. CRIAÇÃO DA DIMENSÃO CLIENTE (DIM_CUSTOMER) - SCD TIPO 2
# Apenas a combinação customer_ID + segment é considerada única para o histórico SCD 2 inicial
cols_cust = ['customer_ID', 'customer_name', 'segment']
df_dim_cliente = df_superstore[cols_cust].drop_duplicates().reset_index(drop=True)

# Geração da CHAVE SUBSTITUTA (PK)
df_dim_cliente.insert(0, 'chave_cliente', df_dim_cliente.index + 1)

# Implementação inicial de colunas SCD Tipo 2
df_dim_cliente['data_inicio_vigencia'] = df_superstore['order_date'].min().strftime('%Y-%m-%d')
df_dim_cliente['data_fim_vigencia'] = None # NULL, pois são registros atuais
df_dim_cliente['versao_atual'] = True

print(f"Dim_Cliente (SCD T2) criada com sucesso. {len(df_dim_cliente)} registros de cliente/segmento.")
df_dim_cliente.head(2)

Dim_Cliente (SCD T2) criada com sucesso. 4873 registros de cliente/segmento.


,chave_cliente,customer_ID,customer_name,segment,data_inicio_vigencia,data_fim_vigencia,versao_atual
0,1,DW-34801,Dianna Wilson,Home Office,2011-01-01,None,True
1,2,AH-101952,Alan Haines,Corporate,2011-01-01,None,True


##Passo 3: Transformação (T) - Tabela Fato

In [ ]:
# 6. CRIAÇÃO DA TABELA FATO (FACT_SALES)

# Cria uma cópia da tabela original para evitar modificar a fonte
df_fato = df_superstore.copy()

# --- MERGE com DIM_TEMPO ---
# Adiciona as chaves de tempo (AAAAMMDD) à Fato
df_fato['chave_tempo_pedido'] = df_fato['order_date'].dt.strftime('%Y%m%d').astype(int)
df_fato['chave_tempo_envio'] = df_fato['ship_date'].dt.strftime('%Y%m%d').astype(int)

# --- MERGE com DIM_LOCALIDADE ---
df_fato = pd.merge(
    df_fato,
    df_dim_localidade,
    on=['city', 'state', 'country', 'region', 'market'],
    how='left'
)

# --- MERGE com DIM_PRODUTO ---
df_fato = pd.merge(
    df_fato,
    df_dim_produto[['chave_produto', 'product_id']],
    on='product_id',
    how='left'
)

# --- MERGE com DIM_CLIENTE ---
# Usamos customer_ID E segment para ligar corretamente, devido ao SCD T2
df_fato = pd.merge(
    df_fato,
    df_dim_cliente[['chave_cliente', 'customer_ID', 'segment']],
    on=['customer_ID', 'segment'],
    how='left'
)

# --- SELEÇÃO FINAL DA TABELA FATO ---
cols_fato = [
    'row_id',
    'order_id',
    'chave_cliente',
    'chave_produto',
    'chave_localidade',
    'chave_tempo_pedido',
    'chave_tempo_envio',
    'sales',
    'quantity',
    'profit',
    'discount',
    'shipping_cost',
    'order_priority',
    'ship_mode'
]

df_fato_vendas = df_fato[cols_fato]
df_fato_vendas = df_fato_vendas.rename(columns={'order_id': 'chave_pedido'})

print(f"Fato_Vendas criada com sucesso. {len(df_fato_vendas)} transações.")
df_fato_vendas.head(2)

Fato_Vendas criada com sucesso. 51290 transações.


,row_id,chave_pedido,chave_cliente,chave_produto,chave_localidade,chave_tempo_pedido,chave_tempo_envio,sales,quantity,profit,discount,shipping_cost,order_priority,ship_mode
0,43411,NI-2011-6340,1,1,1,20110416,20110421,135,14,-90.174,0.7,10.61,Medium,Standard Class
1,19231,ES-2011-4993245,2,2,2,20110408,20110412,108,4,49.560,0.0,9.00,Medium,Standard Class


## Passo 4: Carga (L) no Google BigQuery

In [ ]:
# Mapeamento dos DataFrames para os nomes das tabelas no BigQuery
tables_to_load = {
    "dim_tempo": df_dim_tempo,
    "dim_localidade": df_dim_localidade,
    "dim_produto": df_dim_produto,
    "dim_cliente": df_dim_cliente,
    "fato_vendas": df_fato_vendas
}

# Configuração da carga (Load)
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE", # Sobrescreve a tabela se ela existir
)

print(f"Iniciando a carga de dados no BigQuery (Dataset: {DATASET_ID})...")

for table_name, df in tables_to_load.items():
    table_id = f"{PROJECT_ID}.{DATASET_ID}.{table_name}"

    try:
        # Envia o DataFrame para o BigQuery
        job = client.load_table_from_dataframe(
            df, table_id, job_config=job_config
        )
        job.result() # Espera o job terminar

        print(f"  [SUCESSO] Tabela '{table_name}' carregada. {job.output_rows} linhas.")

    except Exception as e:
        print(f"  [ERRO] Falha ao carregar a tabela '{table_name}': {e}")

print("\nProcesso ETL (Carga) concluído!")

Iniciando a carga de dados no BigQuery (Dataset: project_rota01)...
  [SUCESSO] Tabela 'dim_tempo' carregada. 1468 linhas.
  [SUCESSO] Tabela 'dim_localidade' carregada. 3819 linhas.
  [SUCESSO] Tabela 'dim_produto' carregada. 10292 linhas.
  [SUCESSO] Tabela 'dim_cliente' carregada. 4873 linhas.
  [SUCESSO] Tabela 'fato_vendas' carregada. 51290 linhas.

Processo ETL (Carga) concluído!
